In [2]:
import tweetple

from tweetple import TweetPle

import pandas as pd
import os
import glob
import yaml
import time
import numpy as np

import sys
sys.path.insert(0, '../../../src/utils')
from funcs import *
from tqdm import tqdm
sys.path.insert(0, '../../../src/utils')
from general import *

def unique_tweets(path_read):
    """Get unique tweets"""
    df = read_files(path_read)
    df = df.drop_duplicates(subset='id').reset_index(drop=True)
    df = expand_column(df, 'entities.urls')

    return df


def tweet_types(df, path_save):
    df0 = df[~df.referenced_tweets.isna()]
    m = pd.DataFrame(df0['referenced_tweets'].tolist())
    m = m[0].apply(pd.Series)
    m.replace({'retweeted': 'retweet', 'replied_to': 'reply',
               'quoted': 'quote'}, inplace=True)
    df0 = pd.concat([df0, m['type']], axis=1)
    df1 = df[df.referenced_tweets.isna()]
    df1['type'] = 'tweet'
    df = df0.append(df1).reset_index(drop=True)
    df = df[~df.id.isna()].reset_index(drop=True)
    create_folder(path_save)
    #df.to_parquet(f'{path_save}tweets.parquet')
    return df

def get_path(country):
    base = f'../../../data/03-experiment/{country}/baseline/'
    path_tw = base + '00-raw/influencers/tweets/'
    path_fb = base + '00-raw/influencers/posts/'
    path_likes = f'{base}00-raw/influencers/likes/'
    path_followers = f'{base}00-raw/influencers/followers/'
    path_converstations = f'{base}00-raw/influencers/conversations/'
    path_retweets = f'{base}00-raw/influencers/retweets/'
    path_save =  f'{base}01-preprocess/influencers/'
    return path_tw, path_fb, base, path_likes, path_followers, path_converstations, path_retweets, path_save

# Change country
country = 'KE'
path_tw, path_fb, base, path_likes, path_followers, path_converstations, path_retweets, path_save = get_path(country)

In [6]:
df = pd.read_parquet(f'../../../data/01-characterize/influencers/SA/confirmed_influencers_batch2.parquet')
df.drop_duplicates('id')

,handle,id
0,PhumiekaLiyo,1495782642
1,KAmuses,238478300
2,saaleha,13714272
3,SISONKE_MD,453721930
4,MlamuliSA,1267959115
5,maseogane,184017607
6,innomagagulajr,1006061273417699328
7,Malatjie_,32943963
8,C_liveDj,16517479
9,TSJ_Letlapa,1169197110081851392


In [3]:
data = pd.read_excel(f'../../../data/01-characterize/influencers/{country}/confirmed_batch2.xlsx')
# Cleaning the handle
data.rename({'Username':'URL'}, axis = 1, inplace = True)
data['URL'] = (np.where(data['URL'].str.contains('\?'),
                  data['URL'].str.split('\?').str[0],
                  data['URL']))

data['handle'] = (np.where(data['URL'].str.contains('/'),
                  data['URL'].str.split('/').str[-1],
                  np.nan))
data = data.drop_duplicates('handle')

data = data[['handle', 'URL']]

data.to_parquet(f'../../../data/01-characterize/influencers/{country}/confirmed.parquet', 
                index = False)

# List of confirmed

confirmed = list(data['handle'].astype(str))
len(confirmed)

60

In [10]:
# Check if we have downloaded all posts for all confirmed influencers
from os import listdir
from os.path import isfile, join
mypath = path_tw

onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
onlyfiles = [f.replace('.parquet', '') for f in onlyfiles]
#ids de tweets
#ids_tweets = list(df.id)
ids = list(set(confirmed).difference(onlyfiles))
ids.sort()
len(ids)

2

In [11]:
# If ids is not null, download posts for the missing influencers
bearer_token = "AAAAAAAAAAAAAAAAAAAAAFpgZAEAAAAAbJS59UWzipi32ixd7LHtXov9olo%3D7gxD8Afshgj4munMXHLU08jzRdTpsAh4RZqq7VBofq1wAvkx1T"

## BASELINE DATES
### 6 months
start =  '2022-08-21T00:00:00Z'
end = '2023-02-21T00:00:00Z'

TweetPle.TweetStreamer(ids, bearer_token, path_save = path_tw, start_time = start, end_time = end).main()

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:02<00:00,  1.47s/it]


In [12]:
# Aggregate posts for confirmed influencers
df_final = pd.DataFrame()
ids_final = list(set(confirmed).intersection(onlyfiles))
for name in ids_final:
    df = pd.read_parquet(f'{path_tw}{name}.parquet')
    df_final = pd.concat([df_final, df]).reset_index(drop=True)

In [13]:
# Clean up posts and export to the 01-preprocess folder
df_final = df_final.drop_duplicates(subset='id').reset_index(drop=True)
df_final = expand_column(df_final, 'entities.urls')
df_final = tweet_types(df_final, path_save)

df_final.to_parquet(f'{path_save}tweets_batch2.parquet', index = False)

C:\Users\Dell\AppData\Local\Temp\ipykernel_18044\663669839.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['type'] = 'tweet'
C:\Users\Dell\AppData\Local\Temp\ipykernel_18044\663669839.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df0.append(df1).reset_index(drop=True)


Directory  ../../data/03-experiment/KE/baseline/01-preprocess/influencers/  already exists
